In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import models, transforms
from src.dataset.coco import get_coco
import src.dataset.DatasetReader as DatasetReader
import matplotlib.pyplot as plt
import src.dataset.utils as utils


In [2]:

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(device)


cpu


In [ ]:

transform = transforms.Compose([
    transforms.Resize((224, 224)),  
    transforms.ToTensor(),  
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  
])

dataset = DatasetReader.COCODataset(annotation_file='../data/annotations/annotations/instances_val2017.json',
    image_dir= '../data/val2017/val2017',
    target_classes=set([s.lower() for s in utils.GLOBAL_CLASSES]),
    transform=transform)

dataloader = DataLoader(dataset, batch_size=8, shuffle=True)

plt.imshow(dataset[50][0].permute(1, 2, 0).numpy())

model = models.mobilenet_v3_small(pretrained=True)
model.classifier[3] = nn.Linear(model.classifier[3].in_features, 90)  
model = model.to(device)

#for param in model.features.parameters():
 #   param.requires_grad = False

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.00001)

def train_model(model, dataloader, criterion, optimizer, num_epochs=10):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for i, (images, labels) in enumerate(dataloader):

            #print(type(images))
            #print(type(labels))
            #print(labels)

            images, labels = images.to(device), labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(images)
            # print(type(outputs))
            # print(outputs.shape)

            # print(type(labels))
            # print(labels.shape)
            # print(labels)

            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
            if (i + 1) % 10 == 0:
                print(f"Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(dataloader)}], Loss: {loss.item():.4f}")
        print(f"Epoch [{epoch+1}/{num_epochs}], Average Loss: {running_loss / len(dataloader):.4f}")

train_model(model, dataloader, criterion, optimizer)